In [3]:
#export
import torch
import torch.nn as nn
from fastai.vision.all import *
from fastai.data.all import *
from fastai.distributed import *
import pandas as pd
from pathlib import Path
import time
from modules_th.video_block import * 
from modules_th.inflator import *
from modules_th.triplet_loss import *


In [4]:
import torch
import torch.nn as nn
import torchvision.transforms as T
from fastai.vision.all import *
from fastai.data.all import *
from fastai.distributed import *
import pandas as pd
from pathlib import Path
from modules_th.video_block import * 
from modules_th.inflator import *
from modules_th.triplet_loss import *

In [6]:
#export 
n_lbl = 4
n_el = 2

vid_paths = get_files('/mnt/data/eugeniomarinelli/UCF_Crimes/Videos', extensions='.mp4')
lbls = L(map(parent_label,vid_paths))
             
tfms = [[CreateVideoTensor(l=60,skip=3), RandomCropVid((224,224)),RandomHFlipVid(0.5)],
        [parent_label, Categorize()]]

splits = RandomSplitter(valid_pct=0.2, seed=42)(vid_paths)

splits = [uniformize_dataset(idxs, lbls[idxs] , n_lbl=n_lbl, n_el=n_el) for idxs in splits] #???

dsets = Datasets(vid_paths, tfms, splits=splits)




In [7]:
#export
# Trainig


dls  = dsets.dataloaders( bs=n_el*n_lbl, 
                        shuffle_train=False,
                       after_batch=[IntToFloatTensor(), Normalize.from_stats(*imagenet_stats)])

head, body = inflate(create_head(1024, len(dls.vocab), lin_ftrs=[256])), inflate(create_body(resnet34, cut=-2)) 
learn = Learner(dls, 
#                 NullModel(),
                TLModel(body,head),
#                 loss_func=NullLoss(),
                MixedLoss(alpha=1.0, margin=.2),
#                 splitter=null_splitter,
                splitter=my_splitter,
#                 opt=Optimizer([0], noop),
#                 metrics=lambda *x: 0)
                metrics=tl_accuracy)

In [ ]:
#export
learn.load('UCF_model_2')
learn.fine_tune(8)
learn.save('UCF_model_2')

# Saved models: UCF_model_1 (bs=16- 4lblx4el frames= 30): acc ~ 0.20
            #   UCF_model_2 (bs = 8 4lblx2el, frames = 60 ): acc ~ 0.15

In [10]:
learn.load('../models/UCF_model_2')

FileNotFoundError: [Errno 2] No such file or directory: 'models/../models/UCF_model_2.pth'

In [1]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_triplet_loss.ipynb.
Converted 02_inflator.ipynb.
Converted 03_video_block.ipynb.
Converted 04_charades.ipynb.
Converted 05_ucf.ipynb.
Converted 06_distributed_inflated_NN.ipynb.
Converted 07_read_video_function.ipynb.
Converted Untitled.ipynb.
Converted index.ipynb.


In [1]:
# default_exp ucf_train

In [11]:
torch.cuda.empty_cache()